<a href="https://colab.research.google.com/github/metalnom/Colab_01/blob/master/note03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN

# img2col 구현

In [0]:
import numpy as np

def im2col(image, flt_h, flt_w, out_h, out_w):
  img_h, img_w = image.shape

  cols = np.zeros((flt_h*flt_w, out_h*out_w))

  for h in range(out_h):
    h_lim = h + flt_h
    for w in range(out_w):
      w_lim = w + flt_w
      cols[:, h*out_w + w] = image[h:h_lim, w:w_lim].reshape(-1)

  return cols

In [19]:
img = np.array([[1, 2, 3, 4],
               [5, 6, 7, 8],
               [9, 10, 11, 12],
               [13, 14, 15, 16]])
cols = im2col(img, 2, 2, 3, 3)
print(cols)

[[ 1.  2.  3.  5.  6.  7.  9. 10. 11.]
 [ 2.  3.  4.  6.  7.  8. 10. 11. 12.]
 [ 5.  6.  7.  9. 10. 11. 13. 14. 15.]
 [ 6.  7.  8. 10. 11. 12. 14. 15. 16.]]


In [0]:
# 배열 접근을 줄여 속도 향상

import numpy as np

def im2col(image, flt_h, flt_w, out_h, out_w):
  img_h, img_w = image.shape
  cols = np.zeros((flt_h, flt_w, out_h, out_w))

  for h in range(flt_h):
    h_lim = h + out_h
    for w in range(flt_w):
      w_lim = w + out_w
      cols[h, w, :, :] = image[h:h_lim, w:w_lim]
  
  cols = cols.reshape(flt_h*flt_w, out_h*out_w)

  return cols

In [21]:
cols = im2col(img, 2, 2, 3, 3)
print(cols)

[[ 1.  2.  3.  5.  6.  7.  9. 10. 11.]
 [ 2.  3.  4.  6.  7.  8. 10. 11. 12.]
 [ 5.  6.  7.  9. 10. 11. 13. 14. 15.]
 [ 6.  7.  8. 10. 11. 12. 14. 15. 16.]]


# 배치와 채널을 고려한 img2col

In [0]:
def im2col(images, flt_h, flt_w, out_h, out_w):
  n_bt, n_ch, img_h, img_w = images.shape
  cols = np.zeros((n_bt, n_ch, flt_h, flt_w, out_h, out_w))

  for h in range(flt_h):
    h_lim = h + out_h
    for w in range(flt_w):
      w_lim = w + out_w
      cols[:, :, h, w, :, :] = images[:, :, h:h_lim, w:w_lim]
  
  cols = cols.transpose(1, 2, 3, 0, 4, 5).reshape(
      n_ch * flt_h * flt_w, n_bt * out_h * out_w)
  return cols

In [0]:
img = np.array([[[[1, 2, 3, 4],
               [5, 6, 7, 8],
               [9, 10, 11, 12],
               [13, 14, 15, 16]]]])

In [26]:
cols = im2col(img, 2, 2, 3, 3)
print(cols)

[[ 1.  2.  3.  5.  6.  7.  9. 10. 11.]
 [ 2.  3.  4.  6.  7.  8. 10. 11. 12.]
 [ 5.  6.  7.  9. 10. 11. 13. 14. 15.]
 [ 6.  7.  8. 10. 11. 12. 14. 15. 16.]]


# 패딩과 스트라이딩을 고려

In [0]:
def im2col(images, flt_h, flt_w, out_h, out_w, stride, pad):
  n_bt, n_ch, img_h, img_w = images.shape

  img_pad = np.pad(images, [(0, 0), (0, 0), (pad, pad), (pad, pad)], "constant" )
  print("img_pad\n", img_pad)
  cols = np.zeros((n_bt, n_ch, flt_h, flt_w, out_h, out_w))

  for h in range(flt_h):
    h_lim = h + stride*out_h
    for w in range(flt_w):
      w_lim = w + stride*out_w
      cols[:, :, h, w, :, :] = img_pad[:, :, h:h_lim:stride, w:w_lim:stride]
  
  cols = cols.transpose(1, 2, 3, 0, 4, 5).reshape(n_ch*flt_h*flt_w, n_bt*out_h*out_w)
  return cols

In [43]:
cols = im2col(img, 2, 2, 3, 3, 1, 1)
print(cols)

img_pad
 [[[[ 0  0  0  0  0  0]
   [ 0  1  2  3  4  0]
   [ 0  5  6  7  8  0]
   [ 0  9 10 11 12  0]
   [ 0 13 14 15 16  0]
   [ 0  0  0  0  0  0]]]]
[[ 0.  0.  0.  0.  1.  2.  0.  5.  6.]
 [ 0.  0.  0.  1.  2.  3.  5.  6.  7.]
 [ 0.  1.  2.  0.  5.  6.  0.  9. 10.]
 [ 1.  2.  3.  5.  6.  7.  9. 10. 11.]]


# col2img 구현

In [0]:
import numpy as np

def col2im(cols, img_shape, flt_h, flt_w, out_h, out_w, stride, pad):
  n_bt, n_ch, img_h, img_w = img_shape

  cols = cols.reshape(n_ch, flt_h, flt_w, n_bt, out_h, out_w).transpose(3, 0, 1, 2, 4, 5)
  images = np.zeros((n_bt, n_ch, img_h + 2*pad + stride - 1, img_w + 2*pad + stride - 1))

  for h in range(flt_h):
    h_lim = h + stride * out_h
    for w in range(flt_w):
      w_lim = w + stride * out_w
      images[:, :, h:h_lim:stride, w:w_lim:stride] += cols[:, :, h, w, :, :]
  
  return images[:, :, pad:img_h+pad, pad:img_w+pad]

In [45]:
cols = np.ones((4, 4))
img_shape = (1, 1, 3, 3)
images = col2im(cols, img_shape, 2, 2, 2, 2, 1, 0)
print(images)

[[[[1. 2. 1.]
   [2. 4. 2.]
   [1. 2. 1.]]]]


# 콘볼루션층 구현

In [0]:
import numpy as np

class ConvLayer:
  def __init__(self, x_ch, x_h, x_w, n_flt, flt_h, flt_w, stride, pad):
    self.params = (x_ch, x_h, x_w, n_flt, flt_h, flt_w, stride, pad)

    self.w = wb_width * np.random.randn(n_flt, x_ch, flt_h, flt_w)
    self.b = wb_width * np.random.randn(1, n_flt)

    self.y_ch = n_flt
    self.y_h = (x_h - flt_h + 2*pad) // stride + 1
    self.y_w = (x_w - flt_w + 2*pad) // stride + 1

  def forward(self, x):
    n_bt = x.shape[0]
    x_ch, x_h, x_w, n_flt, flt_h, flt_w, stride, pad = self.params
    y_ch, y_h, y_w = self.y_ch, self.y_h, self.y_w

    self.cols = im2col(x, flt_h, flt_w, y_h, y_w, stride, pad)
    self.w_col = self.w.reshape(n_flt, x_ch*flt_h*flt_w)

    u = np.dot(self.w_col, self.cols).T + self.b
    self.u = u.reshape(n_bt, y_h, y_w, y_ch).transpose(0, 3, 1, 2)
    self.y = np.where(self.u <= 0, 0, self.u)

  def backward(self, grad_y):
    n_bt = grad_y.shape[0]
    x_ch, x_h, x_w, n_flt, flt_h, flt_w, stride, pad = self.params
    y_ch, y_h, y_w = self.y_ch, self.y_h, self.y_w

    delta = grad_y * np.where(self.u <=0, 0, 1)
    delta = delta.transpose(0, 2, 3, 1).reshape(n_bt * y_h * y_w, y_ch)

    grad_w = np.dot(self.cols, delta)
    self.grad_w = grad_w.T.reshape(n_flt, x_ch, flt_h, flt_w)
    self.grad_b = np.sum(delta, axis=0)

    grad_cols = np.dot(delta, self.w_col)
    x_shape = (n_bt, x_ch, x_h, x_w)
    self.grad_x = col2im(grad_cols.T, x_shape, flt_h, flt_w, y_h, y_w, stride, pad)